# Deriving an ANN for predicting customer churn:

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

## Part 1 - Data Preprocessing

### Importing the dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [4]:
dataset.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [5]:
#let us make sure that there are no missing values
dataset.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [6]:
#lets define the predictors and the predicted values: We will not be using the intital 3 columns for our analysis as they are not relevant.
X = dataset.iloc[:, 3:-1].values   #  .values ignores the labels and just stores the associated values 
y = dataset.iloc[:, -1].values   #this is the predicted value (Dependent Variable)

In [7]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [8]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

LabelEncoder can turn [dog,cat,dog,mouse,cat] into [1,2,1,3,2], but then the imposed ordinality means that the average of dog and mouse is cat. Still there are algorithms like decision trees and random forests that can work with categorical variables just fine and LabelEncoder can be used to store values using less disk space.

One-Hot-Encoding has the advantage that the result is binary rather than ordinal and that everything sits in an orthogonal vector space. The disadvantage is that for high cardinality, the feature space can really blow up quickly and you start fighting with the curse of dimensionality. In these cases, I typically employ one-hot-encoding followed by PCA for dimensionality reduction. I find that the judicious combination of one-hot plus PCA can seldom be beat by other encoding schemes. PCA finds the linear overlap, so will naturally tend to group similar features into the same feature.

**Label Encoding the "Gender" column**

In [9]:
#we are using label encoding to encode the labels (Predicted values) of our dataset

from sklearn.preprocessing import LabelEncoder #we are using this as it has just 2 categories
le = LabelEncoder()  #Lets create an object for the label encoder so we can access it.
X[:, 2] = le.fit_transform(X[:, 2])  #Label encoder has the fit_transform function.

In [10]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


**One Hot Encoding the "Geography" column**

In [11]:
#lets 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough') #passthrough ensures that the other columns are passed through untransformed 
X = np.array(ct.fit_transform(X))

In [12]:
print(X)
#one-hot encoding shifts the newly created features to the left of the dataset.

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)#30% data has been used for testing

### Feature Scaling

In [14]:
#Note that all the features need to be scaled for making an effective Neural network
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [15]:
#lets create an object ann.
ann = tf.keras.models.Sequential()  #since keras 2.0 it has been integrated into TensorFlow

'''Sequential specifies to keras that we are creating model sequentially and the output of each
layer we add is input to the next layer we specify.'''

'Sequential specifies to keras that we are creating model sequentially and the output of each\nlayer we add is input to the next layer we specify.'

### Adding the input layer and the first hidden layer

In [16]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))  #rectified linear unit, it is the most frequently used.

### Adding the second hidden layer

In [17]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) #the code is entirely the same, you can change the hyperparameters here to increaset the accuracy

### Adding the output layer

In [18]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
#softmax must be used for the non-binary outputs here.

## Part 3 - Training the ANN

### Compiling the ANN

In [19]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#optimizer=adam allows for stochastic gradient descent.  
#for categorical output  we would use categorical entropy.
#accuarcy accepts an array of multiple metrics and hence uses a square parenthesis.

### Training the ANN on the Training set

In [20]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
219/219 [==============================] - 0s 788us/step - loss: 0.5945 - accuracy: 0.7081
Epoch 2/100
219/219 [==============================] - 0s 833us/step - loss: 0.4826 - accuracy: 0.7920
Epoch 3/100
219/219 [==============================] - 0s 806us/step - loss: 0.4601 - accuracy: 0.7929
Epoch 4/100
219/219 [==============================] - 0s 1ms/step - loss: 0.4492 - accuracy: 0.7941
Epoch 5/100
219/219 [==============================] - 0s 958us/step - loss: 0.4422 - accuracy: 0.7953
Epoch 6/100
219/219 [==============================] - 0s 888us/step - loss: 0.4371 - accuracy: 0.7986
Epoch 7/100
219/219 [==============================] - 0s 879us/step - loss: 0.4328 - accuracy: 0.8003
Epoch 8/100
219/219 [==============================] - 0s 906us/step - loss: 0.4285 - accuracy: 0.8041
Epoch 9/100
219/219 [==============================] - 0s 1ms/step - loss: 0.4239 - accuracy: 0.8101
Epoch 10/100
219/219 [==============================] - 0s 802us/step - loss:

219/219 [==============================] - 0s 788us/step - loss: 0.3376 - accuracy: 0.8600
Epoch 81/100
219/219 [==============================] - 0s 765us/step - loss: 0.3373 - accuracy: 0.8579
Epoch 82/100
219/219 [==============================] - 0s 729us/step - loss: 0.3372 - accuracy: 0.8577
Epoch 83/100
219/219 [==============================] - 0s 820us/step - loss: 0.3373 - accuracy: 0.8591
Epoch 84/100
219/219 [==============================] - 0s 865us/step - loss: 0.3368 - accuracy: 0.8604
Epoch 85/100
219/219 [==============================] - 0s 916us/step - loss: 0.3372 - accuracy: 0.8591
Epoch 86/100
219/219 [==============================] - 0s 888us/step - loss: 0.3375 - accuracy: 0.8581
Epoch 87/100
219/219 [==============================] - 0s 915us/step - loss: 0.3370 - accuracy: 0.8594
Epoch 88/100
219/219 [==============================] - 0s 956us/step - loss: 0.3369 - accuracy: 0.8589
Epoch 89/100
219/219 [==============================] - 0s 781us/step - loss:

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

In [21]:
#note that we have taken the threshold for our churners at 0.5 (50%)
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

#Another thing to note here is that transform is being applied here to the test entry and not fit_transform.

[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [22]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [23]:
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)  #actual values in the row and predicted values in the columns
print('/n')
print(classification_report(y_test, y_pred))

[[2329   87]
 [ 312  272]]
/n
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      2416
           1       0.76      0.47      0.58       584

    accuracy                           0.87      3000
   macro avg       0.82      0.71      0.75      3000
weighted avg       0.86      0.87      0.85      3000



**The complete picture here shows here that the recall(sensitivity) is pretty low here owing to the low number of values for the people who have churned. This makes the model follow more closely the entries for the non-churners and not so much with the ones that have. This kind of problem needs to be solved using oversampler or undersamplers.**